<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V7_pureData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap plotly xgboost --quiet

In [2]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.5 MB/s eta 0:00:00


In [3]:
!pip install ace_tools

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

外部資料讀入

In [6]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1W93D0WRk2GYeWnT8fOsJLwbYlbzJNXJJco1pYUrlZrU/edit?usp=sharing')
worksheet = gsheets.worksheet("external_validation_0820")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1326,NaN,NaN,NaN,1,32,0,93,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,0
1,1317,NaN,NaN,NaN,0,270,0,84,0,40,30,40,10,3.779645,1,0.0,0.0,0.0,1.0,2
2,1301,NaN,NaN,NaN,1,101,0,93,0,30,30,30,0,0.000000,0,0.0,0.0,0.0,1.0,0
3,1329,NaN,NaN,NaN,1,39,0,74,0,5,5,5,0,NaN,0,0.0,0.0,0.0,1.0,0
4,1370,NaN,NaN,NaN,1,6,0,85,0,0,0,0,0,NaN,0,0.0,1.0,0.0,1.0,1


In [7]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,6339.0,1383.704685,767.419894,4.0,1184.0,1265.0,1386.000000,20295.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,6339.0,0.280959,0.449503,0.0,0.0,0.0,1.000000,1.000000
觀察天數,6339.0,210.107588,109.964750,0.0,127.5,235.0,298.000000,365.000000
性別_is_male,6339.0,0.508440,0.499968,0.0,0.0,1.0,1.000000,1.000000
預估年齡,6339.0,78.607825,11.742108,1.0,72.0,81.0,87.000000,124.000000
DNR_flag,6339.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,6339.0,29.695536,31.687105,0.0,0.0,20.0,50.000000,100.000000


In [8]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


訓練資料讀入

In [9]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets

# gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14dXHX3xQlUWoe5uxm2DW-TRKoHee3AcHVVyBbljUMI0/edit?usp=sharing')
# worksheet = gsheets.worksheet("training_data_v1_no_matching")  # 指定分頁名稱

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1t14jZVSsCyh993rZwtrtmzAeXeI4cPGzqGYNnt18jdw/edit?usp=sharing')
worksheet = gsheets.worksheet("training_data_v2")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1216,NaN,NaN,NaN,0,1056,1,81,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,NaN
1,1199,NaN,NaN,NaN,0,1278,1,61,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,NaN
2,1251,NaN,NaN,NaN,0,688,0,68,0,40,40,40,0,NaN,0,0.0,0.0,0.0,1.0,NaN
3,1205,NaN,NaN,NaN,0,1181,1,74,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,1.0
4,1206,NaN,NaN,NaN,0,1166,0,83,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,NaN


In [10]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,27037.0,1500.801457,4160.182209,1.0,1159.0,1220.000000,1317.000000,100463.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,27037.0,0.194992,0.396202,0.0,0.0,0.000000,0.000000,1.000000
觀察天數,27037.0,634.100529,437.553151,0.0,248.0,597.000000,1014.000000,1460.000000
性別_is_male,27037.0,0.485557,0.499801,0.0,0.0,0.000000,1.000000,1.000000
預估年齡,27037.0,79.388801,11.819412,0.0,72.0,82.000000,88.000000,125.000000
DNR_flag,27037.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
ADL_總分_max,27037.0,31.974886,33.277403,0.0,0.0,20.000000,55.000000,100.000000


In [11]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


In [12]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.5].index.tolist()

In [13]:
features

['H01_NUM',
 '性別_is_male',
 '觀察天數',
 '死亡標記',
 '預估年齡',
 'ADL_first_score',
 'ADL_總分_max',
 'DNR_flag',
 'ADL_diff',
 'ADL_明顯惡化',
 'ADL_last_score',
 '意識清醒_max',
 '活動假牙',
 '使用管路',
 '使用呼吸輔具',
 'ADL_std',
 '六個月內住院次數']

In [14]:
dfNew = df[features]

In [15]:
dfNew = dfNew.fillna(0)

In [16]:
from sklearn.base import BaseEstimator, ClassifierMixin

class HybridXGBRF(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_model=None, rf_model=None, alpha=0.5):
        self.xgb_model = xgb_model
        self.rf_model = rf_model
        self.alpha = alpha
        self._init_models()

    def _init_models(self):
        # Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 800, 'subsample': 1.0}
        # "XGBClassifier": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss'),

        self.xgb = self.xgb_model or XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            colsample_bytree=0.8,     # ✅ 降低每棵樹看到的特徵比例 → 提高多樣性
            learning_rate=0.01,       # ✅ 稍微提升學習率搭配更早停止
            max_depth=5,              # ✅ 降低單棵樹複雜度 → 降低過擬合
            n_estimators=200,         # ✅ 總樹數可略減以免累積錯誤
            subsample=1.0,            # ✅ 樣本隨機抽樣 → 提升隨機性
            verbosity=0,
            use_label_encoder=False
        )
        self.rf = self.rf_model or RandomForestClassifier(
            n_estimators=100,
            random_state=42
        )

    def fit(self, X, y):
        self._init_models()  # 每次 fit 要重設模型
        self.xgb.fit(X, y)
        self.rf.fit(X, y)
        return self

    def predict_proba(self, X):
        xgb_prob = self.xgb.predict_proba(X)[:, 1]
        rf_prob = self.rf.predict_proba(X)[:, 1]
        blended = self.alpha * xgb_prob + (1 - self.alpha) * rf_prob
        return np.vstack([1 - blended, blended]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'xgb_model': self.xgb_model,
            'rf_model': self.rf_model,
            'alpha': self.alpha
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self._init_models()  # 重新初始化模型
        return self

In [17]:
# 要移除的欄位，是代表身分標記，以及天數
drop_columns = ['H01_NUM', '觀察天數']

# 丟掉這些欄位
dfNew = dfNew.drop(columns=drop_columns)

In [18]:
dfNew

,性別_is_male,死亡標記,預估年齡,ADL_first_score,ADL_總分_max,DNR_flag,ADL_diff,ADL_明顯惡化,ADL_last_score,意識清醒_max,活動假牙,使用管路,使用呼吸輔具,ADL_std,六個月內住院次數
0,1,0,81,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0,61,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,68,40,40,0,0,0,40,0.0,0.0,1.0,0.0,0.0,0.0
3,1,0,74,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,0,83,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27032,1,0,65,30,30,0,0,0,30,0.0,0.0,0.0,0.0,0.0,0.0
27033,1,0,34,100,100,0,0,0,100,0.0,0.0,1.0,0.0,0.0,109.0
27034,0,0,55,100,100,0,0,0,100,0.0,0.0,1.0,0.0,0.0,178.0
27035,1,0,52,100,100,0,0,0,100,0.0,0.0,1.0,0.0,0.0,31.0


In [19]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, classification_report
)
import plotly.graph_objects as go
import plotly.express as px

In [20]:
# === 資料準備 ===
X = dfNew.drop(columns=['死亡標記'])
y = df['死亡標記']

In [21]:
X_missing_info = X.isnull().sum().to_frame(name='Missing Count')
X_missing_info['Missing Ratio'] = (X_missing_info['Missing Count'] / len(X)).round(4)
X_missing_info = X_missing_info.sort_values(by='Missing Ratio', ascending=True)
X_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_first_score,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_diff,0,0.0
ADL_明顯惡化,0,0.0
ADL_last_score,0,0.0
意識清醒_max,0,0.0
活動假牙,0,0.0


# 開始進行訓練

In [22]:
X.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,27037.0,0.485557,0.499801,0.0,0.0,0.0,1.000000,1.000000
預估年齡,27037.0,79.388801,11.819412,0.0,72.0,82.0,88.000000,125.000000
ADL_first_score,27037.0,20.445131,28.644584,0.0,0.0,5.0,30.000000,100.000000
ADL_總分_max,27037.0,31.974886,33.277403,0.0,0.0,20.0,55.000000,100.000000
DNR_flag,27037.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_diff,27037.0,11.529756,19.766801,0.0,0.0,0.0,15.000000,100.000000
ADL_明顯惡化,27037.0,0.349595,0.476851,0.0,0.0,0.0,1.000000,1.000000
ADL_last_score,27037.0,31.974886,33.277403,0.0,0.0,20.0,55.000000,100.000000
意識清醒_max,27037.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
活動假牙,27037.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [23]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.1 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=b3faf55e318654a3076a3ed311c257bfdb54e5459912cd87fce8c654689276e9
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lifelines import CoxPHFitter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

# 假設 HybridXGBRF 已定義
all_models = {
    #"HybridXGBRF (Our Approach)": HybridXGBRF(alpha=1),
    "HybridXGBRF (Our Approach)": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss', subsample=1.0, verbosity=0),
    "LogisticRegression (max_iter=200)": LogisticRegression(max_iter=200),
    "XGBClassifier": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=3, random_state=42, eval_metric='logloss'),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression (max_iter=1000)": LogisticRegression(max_iter=1000),

    # 🔽 新增未測試模型
    "Ridge": make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='saga', max_iter=1000, random_state=42)),
    "Lasso": make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)),
    "Elastic": make_pipeline(StandardScaler(), LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42)),
}

In [25]:
import copy
from sklearn.base import clone
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted

# Prepare CV and ROC Figure
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fig_roc = go.Figure()
mean_fpr = np.linspace(0, 1, 100)

results = []
trained_models = {}

for model_name, model in all_models.items():
    print(f"▶ Running CV for: {model_name}")
    accs, precs, recalls, f1s, aucs = [], [], [], [], []
    all_cm = np.zeros((2, 2), dtype=int)
    tprs = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model_fold = clone(model)

        try:
            # 嘗試直接 fit（若模型支援 NaN，會成功）
            model_fold.fit(X_train, y_train)

        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️ Missing value detected for {model_name} (fold {fold+1}) — applying median imputation.")

                imputer = SimpleImputer(strategy='median')
                X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
                X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

                model_fold.fit(X_train, y_train)
            else:
                raise e  # 若是其他錯誤就直接拋出

        y_pred = model_fold.predict(X_test)
        y_prob = model_fold.predict_proba(X_test)[:, 1]

        # ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc)

        # Metrics
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        all_cm += confusion_matrix(y_test, y_pred)

    trained_models[model_name] = copy.deepcopy(model_fold)

    # ROC Curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    fig_roc.add_trace(go.Scatter(
        x=mean_fpr, y=mean_tpr, mode='lines',
        name=f"{model_name} (mean AUC={mean_auc:.3f})"
    ))

    results.append({
        'Model': model_name,
        'Accuracy Mean': np.mean(accs),
        'Accuracy Std': np.std(accs),
        'Precision Mean': np.mean(precs),
        'Recall Mean': np.mean(recalls),
        'F1 Score Mean': np.mean(f1s),
        'ROC AUC Mean': np.mean(aucs),
        'ROC AUC Std': np.std(aucs),
        'Confusion Matrix': all_cm
    })

# Add Random Baseline
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    line=dict(dash='dash'), name='Random Baseline'
))

fig_roc.update_layout(
    title="ROC Curve Comparison (Cross-Validation Mean)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    width=800, height=600
)

fig_roc.show()

▶ Running CV for: HybridXGBRF (Our Approach)
▶ Running CV for: LogisticRegression (max_iter=200)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/

▶ Running CV for: XGBClassifier
▶ Running CV for: RandomForestClassifier
▶ Running CV for: LogisticRegression (max_iter=1000)
▶ Running CV for: Ridge
▶ Running CV for: Lasso


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Elastic


In [26]:
import shap

# 訓練模型（完整資料）
xgb_model = all_models["HybridXGBRF (Our Approach)"]
xgb_model.fit(X, y)

# 建立 SHAP 解釋器
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X)

# 計算平均 SHAP 值絕對值（作為重要性）
import numpy as np
shap_abs_mean = np.abs(shap_values).mean(axis=0)

In [27]:
import pandas as pd
import plotly.express as px

# 整理成 DataFrame 並排序
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Mean |SHAP Value|': shap_abs_mean
}).sort_values(by='Mean |SHAP Value|', ascending=False)

# 畫前 20 名
top_n = 20
fig_bar = px.bar(
    importance_df.head(top_n),
    x='Mean |SHAP Value|',
    y='Feature',
    orientation='h',
    title="🎯 Top SHAP Features by Mean |SHAP|",
)
fig_bar.update_layout(yaxis=dict(categoryorder='total ascending'))
fig_bar.show()

In [28]:
import plotly.graph_objects as go

for r in results:
    cm = r['Confusion Matrix']
    model_name = r['Model']

    z = cm
    x_labels = ['Predicted 0', 'Predicted 1']
    y_labels = ['Actual 0', 'Actual 1']

    fig_cm = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        colorscale='Blues',
        text=z,
        texttemplate="%{text}"
    ))

    fig_cm.update_layout(
        title=f"Confusion Matrix - {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        width=500,
        height=500
    )

    fig_cm.show()

In [29]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="ROC AUC Mean", ascending=False).reset_index(drop=True)
results_df

,Model,Accuracy Mean,Accuracy Std,Precision Mean,Recall Mean,F1 Score Mean,ROC AUC Mean,ROC AUC Std,Confusion Matrix
0,XGBClassifier,0.816881,0.001769,0.612544,0.168814,0.264235,0.823522,0.004988,"[[21196, 569], [4382, 890]]"
1,HybridXGBRF (Our Approach),0.815290,0.001783,0.634643,0.124999,0.208734,0.822175,0.005128,"[[21384, 381], [4613, 659]]"
2,LogisticRegression (max_iter=1000),0.805082,0.001028,0.496854,0.019348,0.037203,0.775331,0.006504,"[[21665, 100], [5170, 102]]"
3,LogisticRegression (max_iter=200),0.805082,0.001083,0.492131,0.018210,0.035068,0.775156,0.006454,"[[21671, 94], [5176, 96]]"
4,Lasso,0.805193,0.001174,0.502923,0.018590,0.035824,0.774893,0.006484,"[[21672, 93], [5174, 98]]"
5,Elastic,0.805156,0.001169,0.500362,0.018210,0.035114,0.774573,0.006450,"[[21673, 92], [5176, 96]]"
6,Ridge,0.805267,0.001093,0.509212,0.018210,0.035128,0.774295,0.006435,"[[21676, 89], [5176, 96]]"
7,RandomForestClassifier,0.788031,0.003418,0.440932,0.324545,0.373837,0.766423,0.011121,"[[19595, 2170], [3561, 1711]]"


# 測試外部資料在 XGBoost 模型下的結果

In [30]:
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1326,NaN,NaN,NaN,1,32,0,93,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,0
1,1317,NaN,NaN,NaN,0,270,0,84,0,40,30,40,10,3.779645,1,0.0,0.0,0.0,1.0,2
2,1301,NaN,NaN,NaN,1,101,0,93,0,30,30,30,0,0.000000,0,0.0,0.0,0.0,1.0,0
3,1329,NaN,NaN,NaN,1,39,0,74,0,5,5,5,0,NaN,0,0.0,0.0,0.0,1.0,0
4,1370,NaN,NaN,NaN,1,6,0,85,0,0,0,0,0,NaN,0,0.0,1.0,0.0,1.0,1


In [31]:
ex_X = external[features].drop(columns=['死亡標記'])
ex_y = external['死亡標記']

In [32]:
ex_X = ex_X.fillna(0)

In [33]:
# 丟掉這些欄位
ex_X = ex_X.drop(columns=drop_columns)

In [34]:
ex_X.describe().T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,6339.0,0.508440,0.499968,0.0,0.0,1.0,1.000000,1.000000
預估年齡,6339.0,78.607825,11.742108,1.0,72.0,81.0,87.000000,124.000000
ADL_first_score,6339.0,21.894621,28.481462,0.0,0.0,10.0,35.000000,100.000000
ADL_總分_max,6339.0,29.695536,31.687105,0.0,0.0,20.0,50.000000,100.000000
DNR_flag,6339.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_diff,6339.0,7.800915,16.083193,0.0,0.0,0.0,10.000000,100.000000
ADL_明顯惡化,6339.0,0.258558,0.437877,0.0,0.0,0.0,1.000000,1.000000
ADL_last_score,6339.0,29.695536,31.687105,0.0,0.0,20.0,50.000000,100.000000
意識清醒_max,6339.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
活動假牙,6339.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [35]:
eX_missing_info = ex_X.isnull().sum().to_frame(name='Missing Count')
eX_missing_info['Missing Ratio'] = (eX_missing_info['Missing Count'] / len(ex_X)).round(4)
eX_missing_info = eX_missing_info.sort_values(by='Missing Ratio', ascending=False)
eX_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
ADL_first_score,0,0.0
ADL_總分_max,0,0.0
DNR_flag,0,0.0
ADL_diff,0,0.0
ADL_明顯惡化,0,0.0
ADL_last_score,0,0.0
意識清醒_max,0,0.0
活動假牙,0,0.0


In [36]:
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted
import plotly.subplots as sp

def evaluate_all_models_visual(models: dict, X_val, y_val):
    mean_fpr = np.linspace(0, 1, 100)
    fig_roc = go.Figure()
    results = []

    # 建立混淆矩陣子圖
    num_models = len(models)
    cols = 3
    rows = int(np.ceil(num_models / cols))

    fig_cm = sp.make_subplots(
        rows=rows, cols=cols,
        subplot_titles=list(models.keys()),
        horizontal_spacing=0.15,
        vertical_spacing=0.15
    )

    for i, (model_name, model) in enumerate(models.items()):
        print(f"🔍 Evaluating {model_name}...")

        # 嘗試使用原始資料
        X_input = X_val.copy()
        y_input = y_val

        # 若模型不支援 NaN，則補值
        try:
            # 嘗試呼叫 predict_proba
            _ = model.predict_proba(X_input)
        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️  {model_name} 不支援 NaN，自動補值中...")
                imputer = SimpleImputer(strategy="median")
                X_input = pd.DataFrame(imputer.fit_transform(X_input), columns=X_val.columns)
            else:
                raise e

        # 預測
        y_pred = model.predict(X_input)
        y_prob = model.predict_proba(X_input)[:, 1]

        # 指標
        acc = accuracy_score(y_input, y_pred)
        prec = precision_score(y_input, y_pred)
        rec = recall_score(y_input, y_pred)
        f1 = f1_score(y_input, y_pred)
        auc_val = roc_auc_score(y_input, y_prob)

        # ROC
        fpr, tpr, _ = roc_curve(y_input, y_prob)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0], tpr_interp[-1] = 0.0, 1.0

        fig_roc.add_trace(go.Scatter(
            x=mean_fpr, y=tpr_interp,
            mode='lines',
            name=f"{model_name} (AUC={auc_val:.3f})"
        ))

        # 混淆矩陣
        cm = confusion_matrix(y_input, y_pred)
        row, col = i // cols + 1, i % cols + 1
        fig_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=["Predicted Negative", "Predicted Positive"],
                y=["Actual Negative", "Actual Positive"],
                colorscale='Blues',
                text=cm,
                texttemplate="%{text}",
                showscale=False
            ),
            row=row, col=col
        )

        results.append({
            "Model": model_name,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "ROC AUC": auc_val
        })

    # 隨機基準線
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(dash='dash'),
        name='Random Baseline'
    ))

    fig_roc.update_layout(
        title="ROC Curve Comparison",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800,
        height=600
    )
    fig_roc.show()

    fig_cm.update_layout(
        title="Confusion Matrices of All Models",
        width=400 * cols,
        height=300 * rows,
        showlegend=False
    )
    fig_cm.show()

    # 指標表格
    df_result = pd.DataFrame(results)

    return df_result

In [37]:
# 假設已經訓練完模型並存在 trained_models 中
evaluate_all_models_visual(trained_models, ex_X, ex_y)

🔍 Evaluating HybridXGBRF (Our Approach)...
🔍 Evaluating LogisticRegression (max_iter=200)...
🔍 Evaluating XGBClassifier...
🔍 Evaluating RandomForestClassifier...
🔍 Evaluating LogisticRegression (max_iter=1000)...
🔍 Evaluating Ridge...
🔍 Evaluating Lasso...
🔍 Evaluating Elastic...


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,HybridXGBRF (Our Approach),0.738445,0.732075,0.108928,0.189638,0.789580
1,LogisticRegression (max_iter=200),0.722196,0.642857,0.025267,0.048622,0.735879
2,XGBClassifier,0.745386,0.710327,0.158338,0.258953,0.794002
3,RandomForestClassifier,0.718252,0.497704,0.304323,0.377700,0.722300
4,LogisticRegression (max_iter=1000),0.722038,0.637681,0.024705,0.047568,0.736176
5,Ridge,0.722196,0.666667,0.022459,0.043455,0.735795
6,Lasso,0.722196,0.656250,0.023582,0.045528,0.736066
7,Elastic,0.722196,0.661290,0.023021,0.044493,0.735934
